## Adjust to semantic model measures
This notebook uses Semantic Link Labs to list and add measures

In [1]:
%pip install semantic-link-labs
import sempy_labs as labs
import sempy.fabric as fabric

StatementMeta(, 58dcaae6-353e-4381-80c0-eaae642bafe2, 8, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.1/756.1 kB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 100.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 69.5 MB/s eta 0:00:00
  Attempting uninstall: azure-core
    Found existing installation: azure-core 1.30.2
    Not uninstalling azure-core at /home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages, outside environment /nfs4/pyenv-d9bba4ba-72ee-401e-bde9-97672ebf9c6a
    Can't uninstall 'azure-core'. No files were found to uninstall.
  Attempting uninstall: semantic-link-sempy
    Found existing installation: semantic-link-sempy 0.10.2
    Not uninstalling semantic-link-sempy at /home/trusted-service-user/clust

In [2]:
semanticmodel_name = "AW2020"
workspace_name = "Semantic Showdown"

StatementMeta(, 58dcaae6-353e-4381-80c0-eaae642bafe2, 10, Finished, Available, Finished)

## List tables


In [3]:
# setup connection to Tabular Object Model (TOM) and list tables as example
with labs.tom.connect_semantic_model(dataset=semanticmodel_name, readonly=False, workspace=workspace_name) as tom:
    for t in tom.model.Tables:
        print(t.Name)

StatementMeta(, 58dcaae6-353e-4381-80c0-eaae642bafe2, 11, Finished, Available, Finished)

FactInternetSales
DimCustomer
DimDate
DimProduct
DimSalesTerritory
DimProductCategory
DimProductSubcategory


## List measures

In [11]:
allmeasures = fabric.list_measures(dataset=semanticmodel_name, workspace=workspace_name)
display(allmeasures)

StatementMeta(, 58dcaae6-353e-4381-80c0-eaae642bafe2, 19, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7df47c75-c336-4d05-84bb-9207615f869c)

## Add measures to the model
[Documentation on used functions](https://semantic-link-labs.readthedocs.io/en/stable/sempy_labs.tom.html#sempy_labs.tom.TOMWrapper.add_measure)

Accepted parameters in add_measure function
- table_name
- measure_name
- expression
- format_string
- hidden
- description
- display_folder
- format_string_expression
- data_category
- lineage_tag
- source_lineage_tag
- detail_row_expression


In [28]:
# Define measures as they should be added to the model

measures = [
    {'table_name': 'FactInternetSales', 'measure_name': 'Total Sales', 'expression': 'SUM(FactInternetSales[SalesAmount])'},
    {'table_name': 'FactInternetSales', 'measure_name': 'LetsTryAnotherOne', 'expression': 'SUM(FactInternetSales[SalesAmount])'},
    {'table_name': 'DimProduct', 'measure_name': 'Selected product color', 'expression': "SELECTEDVALUE('DimProduct'[Color])"},
]


StatementMeta(, 58dcaae6-353e-4381-80c0-eaae642bafe2, 36, Finished, Available, Finished)

In [27]:
# Validating if measure already exists in the model

for measure in measures:
    expr = measure["expression"]

    # Check if the expression already exists in allmeasures DataFrame
    duplicate = allmeasures.loc[allmeasures["Measure Expression"] == expr]

    if not duplicate.empty:
        existing = duplicate.iloc[0]
        print(
            f"⚠️ Skipped: A measure with expression '{expr}' already exists as "
            f"'{existing['Table Name']}' - '{existing['Measure Name']}'"
        )
        continue  # Skip to the next measure

    # If measure does not exist, create it
    with labs.tom.connect_semantic_model(dataset=semanticmodel_name, readonly=False, workspace=workspace_name) as tom:
        tom.add_measure(
            table_name=measure["table_name"], 
            measure_name=measure["measure_name"], 
            expression=measure["expression"]
        )
        print(f"✅ Measure {measure['table_name']}.[{measure['measure_name']}] has been created")


StatementMeta(, 58dcaae6-353e-4381-80c0-eaae642bafe2, 35, Finished, Available, Finished)

⚠️ Skipped: A measure with expression 'SUM(FactInternetSales[SalesAmount])' already exists as 'FactInternetSales' - '$ Sales Amount'
✅ Measure FactInternetSales.[AnotherMeasure] has been created
